In [536]:
from manim import *
import pandas as pd
import jupyter_capture_output

video_scene_main = " -v WARNING --disable_caching writing_visualization_main_Scene"
image_scene_main = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s writing_visualization_main_Scene"

video_scene_coords = " -v WARNING --disable_caching writing_visualization_coords_Scene"
image_scene_coords = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s writing_visualization_coords_Scene"

video_scene_pauses = " -v WARNING --disable_caching writing_visualization_pauses_Scene"
image_scene_pauses = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s writing_visualization_pauses_Scene"

In [372]:
# data processing

# total writing data
writing_data = pd.read_csv(f"../external_media/Raw_Sample_Data_Report_Plus_Segs_Report_ERSERO0_segmentiert0.txt", delimiter = "\t", skiprows = 0, skipinitialspace = True).dropna(subset = "seg_path")
# reduce writing data to kinetic energy
kin_energy_writing_data = writing_data.loc[writing_data["seg_path"].str.contains('ERSERO0->kinEnergie')]
# add an integer time to the dataframe
kin_energy_writing_data["integer_time"] = kin_energy_writing_data["time"]*1000
kin_energy_writing_data["integer_time"] = kin_energy_writing_data["integer_time"].astype(int) 

# pause data
pause_data = pd.read_csv(f"../external_media/ERSERO0_kinEnergie.csv", delimiter = ",", skiprows = 0, skipinitialspace = True)
kin_energy_writing_data
pause_data

/tmp/ipykernel_192651/1978636093.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kin_energy_writing_data["integer_time"] = kin_energy_writing_data["time"]*1000
/tmp/ipykernel_192651/1978636093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kin_energy_writing_data["integer_time"] = kin_energy_writing_data["integer_time"].astype(int)


,Unnamed: 0,name,pauses
0,16,E,0.000
1,17,=,0.458
2,80,1,0.453
3,81,/,0.166
4,82,2,0.270
5,19,m,0.438
6,83,v,0.543
7,84,2,0.396


In [534]:
class SegPath(VMobject):
    def __init__(self, center, main_width, seg_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.seg_data = seg_data

        # process writing data
        self.integer_time_data = seg_data["integer_time"]
        self.x_data = seg_data["x"]
        self.y_data = seg_data["y"]
        self.pressure_data = seg_data["pressure"]
        self.seg_name_data = seg_data["seg_name"]

        # set geometry of main frame
        x_data_range = max(self.x_data) - min(self.x_data)
        y_data_range = max(self.y_data) - min(self.y_data)
        print(x_data_range, y_data_range)

        self.main_width = main_width
        self.main_height = float(self.main_width) / x_data_range * y_data_range
        print(self.main_height, self.main_width)

        # main coordinate system
        self.main_npla = NumberPlane(
            x_range = (min(self.x_data)-x_data_range/10, max(self.x_data)+x_data_range/10, 100), y_range = (min(self.y_data)-y_data_range/2.5, max(self.y_data)+y_data_range/2.5, 100), x_length = self.main_width, y_length = self.main_height,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center)
        self.main_rect = Rectangle(width = self.main_width, height = self.main_height, stroke_width = 1, stroke_color = WHITE).move_to(self.center)
        self.main_square = Square(color = WHITE)
        # self.add(self.main_npla)
        self.add(self.main_rect)


    # returns the dot at position i
    def get_dot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            x_i = float(int_time_column["x"].iloc[0])
            y_i = float(int_time_column["y"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.025, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.main_npla.c2p(x_i, y_i, 0))
            else:
                write_dot = Dot(radius = 0.025, color = RED).move_to(self.main_npla.c2p(x_i, y_i, 0))
            return write_dot
            
            

class SegPathCoordinates(VMobject):
    def __init__(self, center, c_width, seg_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.seg_data = seg_data

        # process writing data
        self.integer_time_data = seg_data["integer_time"]
        self.x_data = seg_data["x"]
        self.y_data = seg_data["y"]
        self.pressure_data = seg_data["pressure"]
        self.seg_name_data = seg_data["seg_name"]

        # geometry
        t_data_range = max(self.integer_time_data) - min(self.integer_time_data)
        x_data_range = max(self.x_data) - min(self.x_data)
        y_data_range = max(self.y_data) - min(self.y_data)
        self.c_width = c_width
        self.c_height = 2.5

        # x-coordinate system
        self.cx_npla = NumberPlane(
            x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), 
            y_range = (min(self.x_data)-x_data_range*2.5, max(self.x_data)+x_data_range*2.5, 100), x_length = self.c_width*0.8, y_length = self.c_height/2,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center + self.c_height/4*UP)

        # y-coordinate system
        self.cy_npla = NumberPlane(
            x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), 
            y_range = (min(self.y_data)-y_data_range*2.5, max(self.y_data)+y_data_range*2.5, 100), x_length = self.c_width*0.8, y_length = self.c_height/2,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center)

        # time number line
        time_number_dict = dict(zip([min(self.integer_time_data) + 1000*i for i in range(6)], [i for i in range(6)]))
        print(time_number_dict)
        self.number_line = NumberLine(x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), length = self.c_width*0.8#, include_numbers = True
            ).move_to(self.center - self.c_height/4*UP).add_labels(time_number_dict, font_size = 24).add_tip(tip_width = 0.25, tip_length = 0.25)
        self.add(self.number_line)

        self.c_rect = Rectangle(width = self.c_width, height = self.c_height, stroke_width = 1, stroke_color = WHITE).move_to(self.center)
        x_descriptor = Tex(r"$x$", font_size = 28, color = WHITE).move_to(self.center + self.c_width/2*LEFT + self.c_height/4*UP + 0.375*RIGHT)
        y_descriptor = Tex(r"$y$", font_size = 28, color = WHITE).move_to(self.center + self.c_width/2*LEFT + 0.375*RIGHT)
        t_descriptor = Tex(r"$t$", font_size = 28, color = WHITE).next_to(self.number_line, RIGHT).shift(0.15*UP)
        # self.add(self.cx_npla, self.cy_npla)
        self.add(self.c_rect, x_descriptor, y_descriptor, t_descriptor)


    # x-dot method
    def get_xdot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            x_i = float(int_time_column["x"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.015, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.cx_npla.c2p(int_time, x_i, 0))
            else:
                write_dot = Dot(radius = 0.015, color = RED).move_to(self.cx_npla.c2p(int_time, x_i, 0))
            return write_dot
        

    # y-dot method
    def get_ydot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            y_i = float(int_time_column["y"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.015, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.cy_npla.c2p(int_time, y_i, 0))
            else:
                write_dot = Dot(radius = 0.015, color = RED).move_to(self.cy_npla.c2p(int_time, y_i, 0))
            return write_dot
        


class SegPathPauses(VMobject):
    def __init__(self, center, p_width, p_height, pause_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.pause_data = pause_data

        self.names = pause_data["name"]
        self.current_name = "null"
        self.current_row_i = 0
        self.pauses = pause_data["pauses"]
        self.names_row_dict = dict(zip([*self.names], [i for i in range(len(self.names))]))

        names_ax_dict = dict(zip([i for i in range(len(self.names))], [*self.names])) 
        print(names_ax_dict)

        self.pause_ax = Axes(x_range = [0, len(self.names)-1, 1], y_range = [0, max(self.pauses)+0.1, 0.1], x_length = p_width, y_length = p_height, 
            tips = False
            ).move_to(self.center).add_coordinates(names_ax_dict, font_size = 24)
        self.pause_ax.y_axis.add_numbers(font_size = 24).add_tip(tip_width = 0.25, tip_length = 0.25)
        self.pause_ax_label = Tex("pauses in s", font_size = 24).next_to(self.pause_ax.y_axis, 0.5*UP)
        self.add(self.pause_ax_label)
        self.add(self.pause_ax)
        

    def get_pause(self, name):
        if (name != self.current_name and name in [*self.names]):
            self.current_name = name
            dot = Circle(radius = 0.05, color = BLUE, fill_opacity = 1, stroke_width = 2).move_to(self.pause_ax.c2p(self.current_row_i, self.pauses.iloc[self.current_row_i], 0))
            if (self.current_row_i != 0):
                connector = Line(start = self.pause_ax.c2p(self.current_row_i-1, self.pauses.iloc[self.current_row_i-1], 0), end = self.pause_ax.c2p(self.current_row_i, self.pauses.iloc[self.current_row_i], 0), color = BLUE)
                self.add(connector)
            self.current_row_i += 1
            return dot

In [537]:
%%manim -qh --fps 60 $video_scene_main


class writing_visualization_main_Scene(Scene):
    def construct(self):

        # main path parameters
        main_path_center = np.array([-2.5, -1.5, 0])
        c_path_center = np.array([-2.5, 1.75, 0])
        pauses_path_center = np.array([4.5, 0, 0])
        
        main_path_width = 8
        pauses_height = 5
        pauses_width = 3


        seg_path = SegPath([0, 0, 0], 12, kin_energy_writing_data)
        seg_path_small = SegPath(main_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path)
        seg_path_dot_group = VGroup()

        seg_path_coordinates = SegPathCoordinates(c_path_center, main_path_width, kin_energy_writing_data)
        # self.add(seg_path_coordinates)

        # seg_path_pauses = SegPathPauses(pauses_path_center, pauses_width, pauses_height, pause_data)
        # self.add(seg_path_pauses)

        min_time = min(kin_energy_writing_data["integer_time"])
        max_time = max(kin_energy_writing_data["integer_time"])

        self.wait(7.5)
        for int_time in range(min_time, max_time):

            # writing path dot
            dot = seg_path.get_dot(int_time)
            if dot:
                seg_path_dot_group.add(dot)
                self.add(dot)

            self.wait(1.0/60)
        self.play(FadeOut(seg_path_dot_group), run_time = 15)
        self.play(FadeTransform(seg_path, seg_path_small), run_time = 10)
        self.wait(2.5)
        self.play(FadeIn(seg_path_coordinates), run_time = 10)


Manim Community v0.18.1

4837 1550
3.8453586934050032 12
4837 1550
2.563572462270002 8
{206940: 0, 207940: 1, 208940: 2, 209940: 3, 210940: 4, 211940: 5}


In [538]:
%%manim -qh --fps 60 $video_scene_coords


class writing_visualization_coords_Scene(Scene):
    def construct(self):

        # main path parameters
        main_path_center = np.array([-2.5, -1.5, 0])
        c_path_center = np.array([-2.5, 1.75, 0])
        pauses_path_center = np.array([4.5, 0, 0])
        
        main_path_width = 8
        pauses_height = 5
        pauses_width = 3


        seg_path = SegPath(main_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path)
        seg_path_dot_group = VGroup()

        seg_path_coordinates = SegPathCoordinates(c_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path_coordinates)
        seg_path_coordinates_group = VGroup()

        seg_path_pauses = SegPathPauses(pauses_path_center, pauses_width, pauses_height, pause_data)
        # self.add(seg_path_pauses)

        min_time = min(kin_energy_writing_data["integer_time"])
        max_time = max(kin_energy_writing_data["integer_time"])

        self.wait(7.5)
        for int_time in range(min_time, max_time):

            # writing path dot
            dot = seg_path.get_dot(int_time)
            if dot:
                seg_path_dot_group.add(dot)
                self.add(dot)

            # x/y coordinates dots
            xdot = seg_path_coordinates.get_xdot(int_time)
            ydot = seg_path_coordinates.get_ydot(int_time)
            if xdot:
                seg_path_coordinates_group.add(xdot)
                self.add(xdot)
            if ydot:
                seg_path_coordinates_group.add(ydot)
                self.add(ydot)

            # # pauses dots
            # seg_name = kin_energy_writing_data[kin_energy_writing_data["integer_time"] == int_time]["seg_name"]
            # if len(seg_name) == 1:
            #     seg_path_pauses_dot = seg_path_pauses.get_pause(seg_name.iloc[0]) 
            #     if seg_path_pauses_dot:
            #         self.add(seg_path_pauses_dot)

            self.wait(1.0/60)
        self.play(FadeOut(seg_path_dot_group), FadeOut(seg_path_coordinates_group), run_time = 15)
        self.wait(2.5)
        self.play(FadeIn(seg_path_pauses), run_time = 10)

Manim Community v0.18.1

4837 1550
2.563572462270002 8
{206940: 0, 207940: 1, 208940: 2, 209940: 3, 210940: 4, 211940: 5}
{0: 'E', 1: '=', 2: '1', 3: '/', 4: '2', 5: 'm', 6: 'v', 7: '2'}


In [539]:
%%manim -qh --fps 60 $video_scene_pauses


class writing_visualization_pauses_Scene(Scene):
    def construct(self):

        # main path parameters
        main_path_center = np.array([-2.5, -1.5, 0])
        c_path_center = np.array([-2.5, 1.75, 0])
        pauses_path_center = np.array([4.5, 0, 0])
        
        main_path_width = 8
        pauses_height = 5
        pauses_width = 3


        seg_path = SegPath(main_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path)

        seg_path_coordinates = SegPathCoordinates(c_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path_coordinates)

        seg_path_pauses = SegPathPauses(pauses_path_center, pauses_width, pauses_height, pause_data)
        self.add(seg_path_pauses)

        min_time = min(kin_energy_writing_data["integer_time"])
        max_time = max(kin_energy_writing_data["integer_time"])

        self.wait(7.5)
        for int_time in range(min_time, max_time):

            # writing path dot
            dot = seg_path.get_dot(int_time)
            if dot:
                self.add(dot)

            # x/y coordinates dots
            xdot = seg_path_coordinates.get_xdot(int_time)
            ydot = seg_path_coordinates.get_ydot(int_time)
            if xdot:
                self.add(xdot)
            if ydot:
                self.add(ydot)

            # pauses dots
            seg_name = kin_energy_writing_data[kin_energy_writing_data["integer_time"] == int_time]["seg_name"]
            if len(seg_name) == 1:
                seg_path_pauses_dot = seg_path_pauses.get_pause(seg_name.iloc[0]) 
                if seg_path_pauses_dot:
                    self.add(seg_path_pauses_dot)

            self.wait(1.0/60)
        self.wait(15)

Manim Community v0.18.1

4837 1550
2.563572462270002 8
{206940: 0, 207940: 1, 208940: 2, 209940: 3, 210940: 4, 211940: 5}
{0: 'E', 1: '=', 2: '1', 3: '/', 4: '2', 5: 'm', 6: 'v', 7: '2'}


In [ ]:
# ffmpeg -f concat -i writing_visualization_merge_list.txt -c copy writing_visualization_FULL_1.mp4
# ffmpeg -i writing_visualization_FULL_1.mp4 -r 128 -vf "setpts=0.1*PTS" writing_visualization_FULL_1_5x.mp4

# ffmpeg -f concat -i writing_visualization_merge_list_2.txt -c copy writing_visualization_FULL_2.mp4
# ffmpeg -i writing_visualization_FULL_2.mp4 -r 128 -vf "setpts=0.1*PTS" writing_visualization_FULL_2_5x.mp4